In [1]:
import sqlite3 as db

import pandas as pd
from pycaret.classification import *

from features import analytical_tools
from utils import helper_funtions

In [2]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes

In [3]:
PATH = helper_funtions.get_path()

# Read data

In [4]:
df = pd.read_csv(PATH + 'data/data_validation.csv')

In [5]:
final_rf = load_model(PATH + 'models/RF_model_28-01-2024')

Transformation Pipeline and Model Successfully Loaded


# Preprocesing

In [6]:
# Determine peak times by rounding the time in groups of one hour 
df['Contact_hour'] = pd.to_datetime(df['Contact_hour'], format='%H:%M:%S')
df['Contact_hour_round'] = df['Contact_hour'].apply(lambda x: x.round('60 min'))

df_peak_time = df['Contact_hour_round'].value_counts().head()
df_peak_time = df_peak_time.rename_axis('Hour')
df_peak_time = df_peak_time.reset_index(name='Count')
peak_hours = []
for i in range(len(df_peak_time)):
    peak_hours.append(df_peak_time['Hour'].iloc[i])

for i in range(len(df)):
    if df.loc[i, 'Contact_hour_round'] in peak_hours:
        df.at[i, 'Is_peak_hour'] = 1
    else:
        df.at[i, 'Is_peak_hour'] = 0

del df_peak_time

In [7]:
labels_to_encoders_name = {
    'Contact_channel':'Contact_channel_ENC',
    'Type_products':'Type_products_ENC',
    'Contact_hour_round':'Contact_hour_round_ENC'}

for var_encoder, encoder_name in labels_to_encoders_name.items():
    analytical_tools.label_encoder(var_encoder, encoder_name, df)

# Model implementation

In [8]:
df_implement = predict_model(final_rf, data=df)
df_implement = df_implement.rename({
    'prediction_label':'Sales'
    }, axis=1)

In [9]:
df_implement.head()

,UUID_client,Age,Location,Income,TAX,Previous_sales,Type_products,Contact_channel,Contact_hour,Num_contacts,Satisfaction_score,Contact_hour_round,Is_peak_hour,Contact_channel_ENC,Type_products_ENC,Contact_hour_round_ENC,Sales,prediction_score
0,b80c8fef-a677-5340-85fb-2c162d75df03,51,FL,80000,12000.000000,1,B,Email,1900-01-01 05:30:00,1,4,1900-01-01 06:00:00,1.0,0,1,1,0,0.8712
1,334b6b31-12a2-5bfc-bf4f-870c0954b343,40,CA,82579,12386.849609,5,A,Phone,1900-01-01 16:45:00,2,4,1900-01-01 17:00:00,0.0,2,0,12,1,0.9576
2,c093c7f6-6edf-595e-9539-70de788efbaa,25,NY,58380,8757.000000,3,C,Online Chat,1900-01-01 06:30:00,4,5,1900-01-01 06:00:00,1.0,1,2,1,1,0.9248
3,ff6c5a08-b2fa-56b6-ac00-6830f4074b6a,57,IL,75000,11250.000000,0,A,Email,1900-01-01 05:30:00,3,2,1900-01-01 06:00:00,1.0,0,0,1,0,0.9964
4,afca48e9-59ea-5a28-b8a4-88e0cc6eda1c,61,CA,50049,7507.350098,0,B,Email,1900-01-01 09:00:00,1,4,1900-01-01 09:00:00,0.0,0,1,4,0,0.9737


# Load data to SQLite

In [10]:
connection = db.connect(PATH + 'output/TELEPERFORMANCE.db')

In [11]:
df_implement.to_sql(
    'TechnicalTest_DS_AMT',
    connection,
    if_exists='replace',
    index=False
    )

99

In [12]:
del df, df_implement  # Delete variables to free memory